In [1]:
import pandas as pd
import numpy as np

# Transaction Preprocessing

## E-transfer Transaction Features
- `e_at_risk`
    - General indicator
    - Probability that a message references a species at risk, or parts thereof
    - *could also be a binary flag*
    - *can look at CITES for this list*
- `e_cad_at_risk`
    - Export indicator
    - Probability that message references a species at risk in Canada, or parts thereof (bear, geese, etc.)
    - *could also be a binary flag*
    - *get animal list from fintrac doc and papers*
- `e_role`
    - General indicator
    - Probability that a message references an IWT related role (e.g. poacher, coordinator, supplier, breeder, trader)
    - *could also be a binary flag*
- `e_trad_med`
    - Import indicator
    - Probability that a message references traditional medicine
    - *could also be a binary flag*
 

*Do we want to separate these into TO and FROM?*

## Wire Transfer Transaction Features
- `w_to_country`
    - Import indicator
    - Binary flag
    - 1 if the wire transfer is to a jurisdiction of concern:
        - China
        - Hong Kong
        - South Africa
        - Australia
        - ...
- `w_from_country`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if from a jurisdiction of concern, 0 otherwise
- `w_external_to_animal`
    - Binary flag
    - 1 if from a jurisdiction of concern to an animal related business, 0 otherwise

## Cash Transaction Features
- `c_large_dep`
    - General indicator
    - Binary flag
    - 1 if large deposit from someone involved in international trade or wildlife-related business
    - *tbd what large means*
- `c_large_wthd`
    - General indicator
    - Binary flag
    - 1 if large withdrawal from someone involved in international trade or wildlife-related business
    - *tbd what large means*

*The financial crime academy has some additional specifications for these features*

# General Transaction Features
- `t_to_animal`
    - Import indicator
    - Binary flag
    - 1 if the transaction is to an animal related business
- `t_from_animal`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if the transaction is from an animal related business
- `t_to_animal_large`
    - Import indicator
    - Binary flag
    - 1 if the transaction is from an animal related and is above a certain dollar amount, 0 otherwise
    - *tbd the dollar amount*
- `t_from_animal_large`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if the transaction is from an animal related and is above a certain dollar amount, 0 otherwise
    - *tbd the dollar amount*
- `t_to_shipping`
    - Export indicator
    - Binary flag
    - 1 if the transaction is to someone in shipping/postal/cargo
    - *no label for this type of occupation yet*


In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

In [2]:
emt_data = pd.read_csv('../data/raw/emt.csv')
wire_data = pd.read_csv('../data/raw/wire.csv')
cash_data = pd.read_csv('../data/raw/cash.csv')
kyc_data = pd.read_csv('../data/processed/kyc_processed.csv')

In [3]:
kyc_data

,Name,Gender,Occupation,Age,Tenure,cust_id,occ_wealth,occ_animal,occ_int,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0,0,0,0
1,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,1,0,0,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,1,0,1,0
3,STEPHEN FIGUEROA,male,Optometrist,35.0,17.0,CUST33995820,0,0,0,0
4,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
195784,THOMAS YOUNG,male,Software Developer,46.0,3.0,CUST23014082,0,0,0,0
195785,CASEY JONES,male,Miner,31.0,10.0,CUST17691251,0,0,0,0
195786,NICOLE-CÉCILE LEBLANC,female,Unknown,21.0,3.0,CUST26444112,0,0,0,0
195787,CATHERINE ARMSTRONG,female,Antiques Dealer,58.0,14.0,CUST96567835,1,0,1,0


In [4]:
# First, rename columns to have a consistent schema
cash_data.rename(columns={
    'amount': 'tx_amount',
    }, inplace=True)
emt_data.rename(columns={
    'emt value': 'tx_amount', 
    'id sender': 'cust_id_sender',
    'id receiver': 'cust_id_receiver',
    'name sender': 'name_sender',
    'name receiver': 'name_receiver',
    'emt message': 'tx_message',
    }, inplace=True)
wire_data.rename(columns={
    'wire value': 'tx_amount',
    'id sender': 'cust_id_sender',
    'id receiver': 'cust_id_receiver',
    'name sender': 'name_sender',
    'name receiver': 'name_receiver',
    'country sender': 'country_sender',
    'country receiver': 'country_receiver',
    }, inplace=True)

# Add a new column to each dataframe to indicate the type of transaction
cash_data['tx_type'] = 'cash'
emt_data['tx_type'] = 'emt'
wire_data['tx_type'] = 'wire'

# duplicate all transactions in emt_data and wire_data to have a sender and receiver as the cust_id
emt_data_sender = emt_data.copy()
emt_data_sender.rename(columns={
    'cust_id_sender': 'cust_id',
    'name_sender': 'name',
    'country_sender': 'country',
    }, inplace=True)
emt_data_receiver = emt_data.copy()
emt_data_receiver.rename(columns={
    'cust_id_receiver': 'cust_id',
    'name_receiver': 'name',
    'country_receiver': 'country',
    }, inplace=True)
wire_data_sender = wire_data.copy()
wire_data_sender.rename(columns={
    'cust_id_sender': 'cust_id',
    'name_sender': 'name',
    'country_receiver': 'country',
    }, inplace=True)
wire_data_receiver = wire_data.copy()
wire_data_receiver.rename(columns={
    'cust_id_receiver': 'cust_id',
    'name_receiver': 'name',
    'country_sender': 'country',
    }, inplace=True)
wire_data_sender['direction'] = 'sender'
wire_data_receiver['direction'] = 'receiver'
emt_data_sender['direction'] = 'sender'
emt_data_receiver['direction'] = 'receiver'

# drop columns that are not needed
emt_data_sender.drop(columns=['cust_id_receiver', 'name_receiver'], inplace=True)
emt_data_receiver.drop(columns=['cust_id_sender', 'name_sender'], inplace=True)
wire_data_sender.drop(columns=['cust_id_receiver', 'name_receiver', 'country_sender'], inplace=True)
wire_data_receiver.drop(columns=['cust_id_sender', 'name_sender', 'country_receiver'], inplace=True)

# drop rows where cust_id doesn't begin with 'CUST'
emt_data_sender = emt_data_sender[emt_data_sender['cust_id'].str.startswith('CUST')]
emt_data_receiver = emt_data_receiver[emt_data_receiver['cust_id'].str.startswith('CUST')]
wire_data_sender = wire_data_sender[wire_data_sender['cust_id'].str.startswith('CUST')]
wire_data_receiver = wire_data_receiver[wire_data_receiver['cust_id'].str.startswith('CUST')]

# remerge the dataframes
emt_data = pd.concat([emt_data_sender, emt_data_receiver], ignore_index=True)
wire_data = pd.concat([wire_data_sender, wire_data_receiver], ignore_index=True)

wire_data = wire_data.drop(columns=['name'])
emt_data = emt_data.drop(columns=['name'])

In [7]:
# Add kyc data to tx data
emt_data = emt_data.merge(kyc_data, on='cust_id', how='left')
wire_data = wire_data.merge(kyc_data, on='cust_id', how='left')
cash_data = cash_data.merge(kyc_data, on='cust_id', how='left')

In [8]:
emt_data

,cust_id,tx_message,tx_amount,trxn_id,tx_type,direction,Name,Gender,Occupation,Age,Tenure,occ_wealth,occ_animal,occ_int,label
0,CUST26232205,for the bike u lent me,154.0,WFEZ76031047,emt,sender,JASON GARRISON,male,Luthier,32.0,13.0,0,0,0,0
1,CUST35533148,NaN,518.0,XQJS86205330,emt,sender,ANTHONY ROBERSON,male,Hotelier,48.0,18.0,0,0,0,0
2,CUST59096559,NaN,46.0,WPXP45854083,emt,sender,KEVIN PARK,male,Import/Export Business Owner,34.0,8.0,1,0,1,0
3,CUST69049633,NaN,570.0,OIRZ70883325,emt,sender,ZHU FENG LAN,male,Retail Salesperson,35.0,0.0,0,0,0,0
4,CUST27403977,NaN,480.0,TRNT55099512,emt,sender,IND.DAVID DUNLAP JR.,other,Private Security Company Owner,69.0,14.0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704566,CUST74979363,NaN,119.0,USHN74907347,emt,receiver,WANDA HAYDEN,female,Cashier,27.0,1.0,0,0,0,0
704567,CUST68693554,NaN,208.0,VXES44436032,emt,receiver,MICHAEL CRUZ,male,Unknown,35.0,0.0,0,0,0,0
704568,CUST90504001,Fox racing motocross gear,150.0,LTUK21435620,emt,receiver,MR. ERIC WALTERS,male,Real Estate Broker,29.0,5.0,1,0,1,0
704569,CUST99824006,NaN,270.0,OIRO35201076,emt,receiver,JOHN HUNT JR.,male,Jewelry Dealer,33.0,2.0,1,0,1,0


In [14]:
emt_data['t_to_animal'] = np.where((emt_data['occ_animal']==1) & (emt_data['direction']=='receiver'), 1, 0)
wire_data['t_to_animal'] = np.where((wire_data['occ_animal']==1) & (wire_data['direction']=='receiver'), 1, 0)
cash_data['t_to_animal'] = np.where(cash_data['occ_animal']==1, 1, 0)

emt_data['t_from_animal'] = np.where((emt_data['occ_animal']==1) & (emt_data['direction']=='sender'), 1, 0)
wire_data['t_from_animal'] = np.where((wire_data['occ_animal']==1) & (wire_data['direction']=='sender'), 1, 0)
cash_data['t_from_animal'] = 0

threshold = 1000
emt_data['t_to_animal_large'] = np.where((emt_data['occ_animal']==1) & (emt_data['direction']=='receiver') & (emt_data['tx_amount'] > threshold), 1, 0)
wire_data['t_to_animal_large'] = np.where((wire_data['occ_animal']==1) & (wire_data['direction']=='receiver') & (wire_data['tx_amount'] > threshold), 1, 0)
cash_data['t_to_animal_large'] = np.where((cash_data['occ_animal']==1) & (cash_data['tx_amount'] > threshold), 1, 0)

emt_data['t_from_animal_large'] = np.where((emt_data['occ_animal']==1) & (emt_data['direction']=='sender') & (emt_data['tx_amount'] > threshold), 1, 0)
wire_data['t_from_animal_large'] = np.where((wire_data['occ_animal']==1) & (wire_data['direction']=='sender') & (wire_data['tx_amount'] > threshold), 1, 0)
cash_data['t_from_animal_large'] = 0

emt_data['t_to_int'] = np.where((emt_data['occ_int']==1) & (emt_data['direction']=='receiver'), 1, 0)
wire_data['t_to_int'] = np.where((wire_data['occ_int']==1) & (wire_data['direction']=='receiver'), 1, 0)
cash_data['t_to_int'] = np.where(cash_data['occ_int']==1, 1, 0)

emt_data['t_from_int'] = np.where((emt_data['occ_int']==1) & (emt_data['direction']=='sender'), 1, 0)
wire_data['t_from_int'] = np.where((wire_data['occ_int']==1) & (wire_data['direction']=='sender'), 1, 0)
cash_data['t_from_int'] = 0

In [15]:
emt_data

,cust_id,tx_message,tx_amount,trxn_id,tx_type,direction,Name,Gender,Occupation,Age,...,occ_wealth,occ_animal,occ_int,label,t_to_animal,t_from_animal,t_to_animal_large,t_from_animal_large,t_to_int,t_from_int
0,CUST26232205,for the bike u lent me,154.0,WFEZ76031047,emt,sender,JASON GARRISON,male,Luthier,32.0,...,0,0,0,0,0,0,0,0,0,0
1,CUST35533148,NaN,518.0,XQJS86205330,emt,sender,ANTHONY ROBERSON,male,Hotelier,48.0,...,0,0,0,0,0,0,0,0,0,0
2,CUST59096559,NaN,46.0,WPXP45854083,emt,sender,KEVIN PARK,male,Import/Export Business Owner,34.0,...,1,0,1,0,0,0,0,0,0,1
3,CUST69049633,NaN,570.0,OIRZ70883325,emt,sender,ZHU FENG LAN,male,Retail Salesperson,35.0,...,0,0,0,0,0,0,0,0,0,0
4,CUST27403977,NaN,480.0,TRNT55099512,emt,sender,IND.DAVID DUNLAP JR.,other,Private Security Company Owner,69.0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704566,CUST74979363,NaN,119.0,USHN74907347,emt,receiver,WANDA HAYDEN,female,Cashier,27.0,...,0,0,0,0,0,0,0,0,0,0
704567,CUST68693554,NaN,208.0,VXES44436032,emt,receiver,MICHAEL CRUZ,male,Unknown,35.0,...,0,0,0,0,0,0,0,0,0,0
704568,CUST90504001,Fox racing motocross gear,150.0,LTUK21435620,emt,receiver,MR. ERIC WALTERS,male,Real Estate Broker,29.0,...,1,0,1,0,0,0,0,0,1,0
704569,CUST99824006,NaN,270.0,OIRO35201076,emt,receiver,JOHN HUNT JR.,male,Jewelry Dealer,33.0,...,1,0,1,0,0,0,0,0,1,0


In [16]:
emt_data.to_parquet('../data/processed/emt.parquet')
wire_data.to_parquet('../data/processed/wire.parquet')
cash_data.to_parquet('../data/processed/cash.parquet')